<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [624]:
pip install pymorphy2[fast]

## **Импорт**

In [625]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [626]:
from IPython.display import SVG, display, HTML

In [627]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [628]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [629]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [630]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv		neuroleptic_antidepr2.txt
annotation_table.csv	neuroleptic_antidepr2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt.2
annotation_table.csv.2	rakpobedim_ru2.txt
content			rakpobedim_ru2.txt.1
hv-info_gepatit2.txt	rakpobedim_ru2.txt.2
hv-info_gepatit2.txt.1	sample_data
hv-info_gepatit2.txt.2
.shortcut-targets-by-id
MyDrive
.file-revisions-by-id
.Trash


In [631]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [632]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-25 18:20:04--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.3’

neuroleptic_antidep 100%[===================>]  11.12M  43.5MB/s    in 0.3s    

2021-05-25 18:20:05 (43.5 MB/s) - ‘neuroleptic_antidepr2.txt.3’ saved [11655409/11655409]

--2021-05-25 18:20:05--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [633]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-25 18:20:08--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-25 18:20:08 ERROR 404: Not Found.

--2021-05-25 18:20:08--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.3’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.

In [634]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [635]:
posts = my_txt.split('----')

In [636]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [637]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [638]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [639]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [640]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [641]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv		hv-info_gepatit2.txt.1	     rakpobedim_ru2.txt
annotation_table.csv	hv-info_gepatit2.txt.2	     rakpobedim_ru2.txt.1
annotation_table.csv.1	hv-info_gepatit2.txt.3	     rakpobedim_ru2.txt.2
annotation_table.csv.2	neuroleptic_antidepr2.txt    rakpobedim_ru2.txt.3
annotation_table.csv.3	neuroleptic_antidepr2.txt.1  sample_data
content			neuroleptic_antidepr2.txt.2
hv-info_gepatit2.txt	neuroleptic_antidepr2.txt.3


In [642]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [643]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [644]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [645]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [646]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [647]:
anns_df.SENT_NUM[18]

1.0

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

Добавим размеченные данные, и посмотрим, что получилось.

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [1194]:
# df = anns_df.iloc[:15000,:]
# df = anns_df.iloc[:17000,:]
df = anns_df.iloc[:18000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [1195]:
len(df.loc[df['NER'] == 1])

629

Из них местоимений (прономинальные анафоры).

In [1196]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [1197]:
len(df.loc[df['NER'] == 2])

232

In [1198]:
len(df.loc[(df['NER'].isin([2,3])) & (df['COREFERENCE_CLUSTER']==0)])

135

In [1199]:
df.loc[df['COREFERENCE_CLUSTER']==0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,"[0.28649247, 0.021926237, 0.16573118, -0.31589...","[0.28649247, 0.021926237, 0.16573118, -0.31589...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,"[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.28649247, 0.021926237, 0.16573118, -0.31589...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
5,не,0,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,"[0.8497808, -0.27083784, -0.85034764, 0.283257...","[0.49640426, -0.16352178, 0.77424294, -0.04560...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
6,знаю,0,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.49640426, -0.16352178, 0.77424294, -0.04560...","[0.49640426, -0.16352178, 0.77424294, -0.04560...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
7,стоило,0,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.20211942, 0.48525754, 0.60634243, 0.258057,...","[0.49640426, -0.16352178, 0.77424294, -0.04560...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,"[1.1943446, 0.9506033, 0.42569792, 0.5044286, ...","[0.73074543, 0.4006274, 0.44870138, 0.22857817...","[0.29677403, 0.29665908, 0.074057706, -0.13029..."
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,"[0.5425904, 0.44698355, 0.7765123, 0.0591632, ...","[1.1943446, 0.9506033, 0.42569792, 0.5044286, ...","[0.29677403, 0.29665908, 0.074057706, -0.13029..."
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,"[0.035110243, 0.27989584, -0.74700576, 0.05364...","[0.041421246, -0.14015144, 0.97275096, 0.41049...","[0.29677403, 0.29665908, 0.074057706, -0.13029..."
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,"[0.1820361, 0.34449643, 0.63227713, -0.1239443...","[-0.056366205, -0.24994576, 0.81406647, 0.2762...","[0.29677403, 0.29665908, 0.074057706, -0.13029..."


In [1200]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [1201]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.342703, 0.88926464, 0.48134792, 0.004741775...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.22977723, 0.89944017, 0.73037416, -0.066387...","[0.342703, 0.88926464, 0.48134792, 0.004741775...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.4941427, 0.8627658, 0.7498076, -0.12329789,...","[0.22977723, 0.89944017, 0.73037416, -0.066387...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.30741495, 0.61254, 0.6675189, 0.5178406, -...","[0.10039262, 0.08739526, 0.15280613, 0.6532856...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.2068827, 0.63091844, 0.79407805, -0.0108524...","[0.3607113, 0.7079143, 0.25857353, 0.024229225...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,0,21,4.0,2.0,137.0,NaN,NaN,masc,NaN,VERB,ROOT,вел,NaN,NaN,masc,VERB,1,136.0,sing,"[1.1326776, 0.07563388, 1.420677, 0.4693263, 0...","[1.1326776, 0.07563388, 1.420677, 0.4693263, 0...","[0.19847971, 0.26420596, 0.21309294, -0.127801..."
17929,соф,0,21,4.0,3.0,148.0,inan,gent,femn,NaN,NOUN,nmod,форума,inan,gent,masc,NOUN,1,148.0,plur,"[0.75306904, 0.17051537, 0.26965442, 0.0081388...","[1.5982186, 0.39864564, 0.83767974, -0.7647795...","[0.19847971, 0.26420596, 0.21309294, -0.127801..."
17930,даке,0,21,4.0,3.0,149.0,inan,datv,femn,NaN,NOUN,nmod,соф,inan,gent,femn,NOUN,1,149.0,sing,"[0.53287816, -0.32071522, 0.20030412, 0.264107...","[0.75306904, 0.17051537, 0.26965442, 0.0081388...","[0.19847971, 0.26420596, 0.21309294, -0.127801..."
17937,соф,0,21,4.0,3.0,156.0,inan,gent,femn,NaN,NOUN,nsubj,вел,NaN,NaN,masc,VERB,1,156.0,plur,"[0.75306904, 0.17051537, 0.26965442, 0.0081388...","[1.1326776, 0.07563388, 1.420677, 0.4693263, 0...","[0.19847971, 0.26420596, 0.21309294, -0.127801..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [1202]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.342703, 0.88926464, 0.48134792, 0.004741775...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.2650704, 0.33247495, 0.101711586, -0.213403..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.1149994, 0.68011516, 0.34320793, 0.11002747...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.39889228, 0.3012593, 0.008999281, -0.007618..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.10692825, 0.83226407, 0.6151822, -0.3002539...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.276279, 0.4616099, 0.09003552, -0.085440174..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.4806179, 1.1054465, 0.19967584, -0.21057804...","[0.28649247, 0.021926237, 0.16573118, -0.31589...","[0.2527346, 0.21567428, 0.12164227, -0.1132859..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.33665493, 0.43112108, 0.8293827, -0.0707395...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.2527346, 0.21567428, 0.12164227, -0.1132859..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.44160134, 0.9263274, 0.5838932, 0.1578917, ...","[-0.119601294, 1.1240623, 0.06137822, 0.774739...","[0.23610201, 0.26355082, 0.13468653, -0.063570..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.10483937, 0.00844331, -1.192633, 0.3754402...","[-0.10483937, 0.00844331, -1.192633, 0.3754402...","[0.23610201, 0.26355082, 0.13468653, -0.063570..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.18633583, 0.506002, 0.8980405, -0.3024382,...","[0.24971649, 0.33581775, 0.7006913, -0.3943021...","[0.26055804, 0.44826156, 0.2113294, 0.08307561..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.25003046, 1.0162405, 0.28071764, 0.02297155...","[0.25003046, 1.0162405, 0.28071764, 0.02297155...","[0.34120727, 0.40474972, -0.039535347, -0.1382..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.132557, 0.56595874, 0.28055114, -0.42701617...","[1.4986717, 0.806077, -0.00045111726, -0.71931...","[0.34120727, 0.40474972, -0.039535347, -0.1382..."


In [1203]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [1204]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [1205]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [1206]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [1207]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [1208]:
df.TOKEN[1]

'препарат'

In [1209]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [1210]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.491172,0.557671,0.940444
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.572827,0.557671,0.940444
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.745660,0.854661,0.940444
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.763700,0.581501,0.940444
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.710837,0.623175,0.940444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,0.429206,0.576195,0.980421
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,0.595870,0.414676,0.980421
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,0.474702,0.496031,0.980421
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,0.525111,0.513144,0.980421


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [1211]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [1212]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [1213]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.737562,0.497892,0.987713
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.524921,0.507669,0.985017
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.474702,0.538696,0.868820
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.432692,0.507556,0.982200
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.474702,0.421938,0.982200
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.469245,0.501431,0.984062
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.551974,0.485895,0.942855
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.480284,0.587294,0.932710
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.618942,0.493082,0.762823
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.464741,0.517337,0.870350


In [1214]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.446693,0.487114,0.940444
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.548614,0.487114,0.940444
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.529863,0.487114,0.940444
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.501839,0.521449,0.940444
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.552369,0.521449,0.940444
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.521924,0.521449,0.940444


In [1215]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [1216]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.5132244229316711, 0.5802256464958191, 0.9404439926147461],
      dtype=object)

In [1217]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [1218]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT


In [1219]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [1289]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 540.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 2.0, 679.0, 0.0, 0.0, 0.0, 1483.0, 0.0, 1750.0, 2031.0, 2053.0, 0.0, 0.0, 0.0, 3125.0, 3270.0, 3619.0, 5.0, 114.0, 174.0, 365.0, 0.0, 527.0, 10.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 239.0, 256.0, 270.0, 270.0, 2.0, 2.0, 447.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1455.0, 1516.0, 1609.0, 1657.0, 1722.0, 1770.0, 0.0, 1905.0, 1984.0, 0.0, 0.0, 2071.0, 0.0, 164.0, 0.0, 410.0, 534.0, 0.0, 0.0, 707.0, 707.0, 801.0, 0.0, 871.0, 881.0, 1350.0, 0.0, 0.0, 0.0, 122.0, 165.0, 263.0, 513.0, 556.0, 654.0, 813.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1867.0, 2138.0, 2295.0, 305.0, 0.0, 0.0, 0.0, 0.0, 752.0]
---
[2.0, 536.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 183.0, 183.0, 183.0, 388.0, 446.0, 686.0, 778.0, 987.0, 987.0, 1490.0, 1582.0, 1750.0, 2031.0, 2053.0, 2344.0, 3.0, 3.0, 3125.0, 3270.0, 

In [1290]:
rule_eval(my_clusters, ann_clusters)

52  true positive
26  false positive
14  true negative
28  false negative
---------
0.6666666666666666  presicion
0.65  recall
0.6582278481012659  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [1222]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [1223]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1, -1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [1224]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'неоднородность', 'душу', 'доходов', 'систем', 'населения',
        'страхования', 'регионах', 'продолжать', 'версиях', 'ранних'],
       dtype='object', length=2982),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [1225]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,-1,3,0,1,5,11,199,0,2,2,1,3,2.0,1,0.523156,0.373665,0.983563
566,он,1,1,9.0,3.0,566.0,-1,0,0,1,5,12,231,-1,-1,0,0,3,536.0,1,0.530008,0.414552,0.983563
587,он,2,2,1.0,0.0,13.0,-1,0,0,1,5,12,235,-1,-1,0,0,3,11.0,1,0.530008,0.557162,0.983563
594,он,2,2,1.0,1.0,20.0,-1,0,0,1,5,12,242,-1,-1,-1,0,3,11.0,1,0.530008,0.614968,0.983563
736,него,2,2,3.0,5.0,162.0,-1,1,0,1,5,10,293,-1,-1,-1,0,3,108.0,1,0.544124,0.571509,0.982723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,2295.0,1,0.544124,0.492059,0.975054
14967,он,16,16,1.0,4.0,33.0,-1,0,0,1,5,12,2504,-1,-1,0,0,3,0.0,1,0.530008,0.649842,0.975054
16328,они,18,18,6.0,2.0,207.0,-1,0,-1,1,5,12,2507,-1,-1,-1,9,3,0.0,0,0.471507,0.527720,0.949666
16391,его,18,18,6.0,5.0,270.0,-1,3,0,1,5,1,1588,-1,-1,-1,5,3,0.0,1,0.523156,0.579209,0.934851


In [1226]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,1,0.745660,0.854661,0.940444
3,рисполепт,1,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0,0.763700,0.581501,0.940444
4,сперидан,1,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,1,0.710837,0.623175,0.940444
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.513224,0.580226,0.940444
142,рисполепта,1,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,1,0.737562,0.482345,0.940444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,21,21,4.0,2.0,137.0,-1,-1,0,-1,0,0,2961,-1,-1,0,0,1,136.0,1,0.513949,0.415904,0.976048
17929,соф,21,21,4.0,3.0,148.0,0,1,1,-1,1,21,2964,0,1,0,1,1,148.0,0,0.520681,0.528626,0.976048
17930,даке,21,21,4.0,3.0,149.0,0,2,1,-1,1,21,2945,0,1,1,1,1,149.0,1,0.509744,0.491247,0.976048
17937,соф,21,21,4.0,3.0,156.0,0,1,1,-1,1,12,2961,-1,-1,0,0,1,156.0,0,0.520681,0.415904,0.976048


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [1291]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] > 1 )], test_size=0.2)

In [1292]:
train = train.reset_index()

In [1293]:
test = test.reset_index()

In [1294]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,15418,вакцину,16,16,11.0,1.0,484.0,0,3,1,-1,1,1,1013,-1,-1,-1,0,2,0.0,1,0.617218,0.466060,0.863575
1,11867,таб,11,11,4.0,0.0,63.0,-1,-1,-1,-1,-1,2,2172,-1,-1,-1,7,2,61.0,-1,0.624762,0.567944,0.958702
2,9635,препараты,7,7,4.0,6.0,1096.0,0,0,0,-1,1,12,619,-1,-1,-1,13,2,0.0,0,0.520919,0.361427,0.981763
3,6927,препарат,5,5,23.0,11.0,729.0,0,0,0,-1,1,4,705,-1,-1,2,0,2,659.0,1,0.572827,0.465284,0.982200
4,5765,прперетов,4,4,5.0,5.0,212.0,0,1,0,-1,1,21,1189,0,0,1,1,2,0.0,0,0.609474,0.431497,0.982200


Делаем наборы для обучения моделей.

In [1295]:
X, y, X_test, y_test = train.drop(['HEAD_VECT', 'NUMBER', 'TOKEN',  'GENDER', 'POS', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE',   'DEPENDENCY',   'HEAD_ANIMACY',	'HEAD_CASE',\
                                   	'HEAD_GENDER',  'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['HEAD_VECT', 'NUMBER','TOKEN','GENDER', 'POS', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', \
                                      'CASE',  'DEPENDENCY',     'HEAD_ANIMACY',	'HEAD_CASE',\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18],\
                                       test.COREFERENCE_CLUSTER

In [1296]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]
X

,index,POST_NUM,SENT_NUM,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,594,1.0,1.0,20.0,3.0,11.0,0.530008,0.983563
1,8714,0.0,13.0,175.0,3.0,164.0,0.523156,0.989174
2,7169,30.0,1.0,971.0,2.0,0.0,0.576297,0.987886
3,6260,3.0,2.0,62.0,3.0,41.0,0.523156,0.982200
4,2055,5.0,2.0,187.0,3.0,183.0,0.530008,0.865241
...,...,...,...,...,...,...,...,...
274,9105,1.0,16.0,566.0,3.0,0.0,0.403448,0.984062
275,11825,0.0,2.0,21.0,3.0,0.0,0.403901,0.975739
276,9908,6.0,13.0,1369.0,2.0,0.0,0.494684,0.979373
277,3452,19.0,0.0,1584.0,3.0,1582.0,0.541991,0.911272


In [1297]:
y

0        11.0
1       164.0
2         0.0
3        41.0
4       183.0
        ...  
274       0.0
275       0.0
276       0.0
277    1582.0
278       0.0
Name: COREFERENCE_CLUSTER, Length: 279, dtype: float64

Обучаем модели.

In [1298]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.5857142857142857

In [1299]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6142857142857143

In [1300]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6285714285714286

In [1301]:

regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [1302]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.37142857142857144

In [1303]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6428571428571429

In [1304]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [1305]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [1306]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [1307]:
# my_eval(list(naive_pred), list(y_test))

In [1308]:
my_eval(list(pred_knn), list(y_test))

20  true positive
21  false positive
25  true negative
4  false negative
---------
0.4878048780487805  presicion
0.8333333333333334  recall
0.6153846153846153  f-score


In [1309]:
my_eval(list(pred_entropy), list(y_test))

17  true positive
27  false positive
26  true negative
0  false negative
---------
0.38636363636363635  presicion
1.0  recall
0.5573770491803279  f-score


In [1310]:
my_eval(list(pred), list(y_test))

15  true positive
29  false positive
26  true negative
0  false negative
---------
0.3409090909090909  presicion
1.0  recall
0.5084745762711864  f-score


In [1311]:
my_eval(list(pred_randf), list(y_test))

18  true positive
24  false positive
26  true negative
2  false negative
---------
0.42857142857142855  presicion
0.9  recall
0.5806451612903225  f-score


In [1312]:
my_eval(list(pred_regr), list(y_test))

2  true positive
68  false positive
0  true negative
0  false negative
---------
0.02857142857142857  presicion
1.0  recall
0.05555555555555556  f-score


In [1249]:
my_eval(list(pred_svm), list(y_test))

0  true positive
0  false positive
25  true negative
45  false negative
---------
0  presicion
0.0  recall
0  f-score


In [1250]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [1251]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.491172,0.940444
1,1.0,0,0.0,0.572827,0.940444
2,2.0,1,2.0,0.745660,0.940444
3,3.0,1,3.0,0.763700,0.940444
4,4.0,1,4.0,0.710837,0.940444
...,...,...,...,...,...
17995,214.0,0,0.0,0.429206,0.980421
17996,215.0,0,0.0,0.595870,0.980421
17997,216.0,0,0.0,0.474702,0.980421
17998,217.0,0,0.0,0.525111,0.980421


In [1252]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

120

In [1253]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.525554,0.434467,0.987713
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.572827,0.854661,0.987713
429,препарат,1,1,6.0,0.0,429.0,0,0,0,-1,1,12,179,-1,-1,-1,0,2,2.0,1,0.572827,0.554474,0.987713
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.620432,0.517389,0.987713
454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.572827,0.505548,0.983563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.523156,0.542085,0.788551
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.544124,0.492059,0.702617
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.544124,0.596017,0.632499
14906,него,15,15,19.0,3.0,2314.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,2295.0,1,0.544124,0.492059,0.975054


# **ПОПАРНЫЙ МЕТОД**

In [1313]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [1471]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]
  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in all_antecedents.values:
      

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                              + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                              + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter != 1:

          pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                               + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                               + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

757
---
1.0 0.0 0


(798, 798, 798)

In [1472]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [1473]:
pair_anaphors[0]

[1.0,
 0.0,
 9.0,
 0,
 1,
 0.5523688793182373,
 0.5214487314224243,
 0.9404439926147461]

In [1474]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

88

In [1475]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.513224,0.580226,0.940444


In [1476]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.789562,0
1,0.768011,0
2,0.722639,0
3,0.721485,0
4,0.711721,0
...,...,...
793,0.999806,0
794,0.999723,0
795,0.999646,0
796,0.999700,0


In [1477]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
26,0.703727,1
27,0.875782,1
33,0.999988,1
36,0.990420,1
38,0.994029,1
...,...,...
643,0.999993,1
651,0.999982,1
669,0.999998,1
684,0.999999,1


In [1478]:
# train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
# train = train.reset_index()
# test = test.reset_index()

train, test = train_test_split(pair_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()


In [1479]:
train

,index,cosine,rel
0,143,0.999996,1
1,648,0.985774,0
2,393,0.699796,0
3,113,0.747874,0
4,579,0.989755,0
...,...,...,...
553,399,0.699510,0
554,404,0.699303,0
555,516,0.990259,0
556,345,0.725290,0


In [1480]:
train.iloc[:1, 1:2]

,cosine
0,0.999996


In [1481]:
test.iloc[:1, 1:2]

,cosine
0,0.747925


In [1482]:
X, y, X_test, y_test = train.iloc[:, 1:2], \
                    train.rel, test.iloc[:, 1:2], test.rel

In [1483]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [1484]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
240  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [1485]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
21  true positive
1  false positive
212  true negative
6  false negative
---------
0.9545454545454546  presicion
0.7777777777777778  recall
0.8571428571428572  f-score


In [1486]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
20  true positive
5  false positive
208  true negative
7  false negative
---------
0.8  presicion
0.7407407407407407  recall
0.7692307692307692  f-score


In [1487]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
20  true positive
5  false positive
208  true negative
7  false negative
---------
0.8  presicion
0.7407407407407407  recall
0.7692307692307692  f-score


In [1488]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
20  true positive
5  false positive
208  true negative
7  false negative
---------
0.8  presicion
0.7407407407407407  recall
0.7692307692307692  f-score


In [1271]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [1272]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [1273]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,0,0,0,-1,1,1,0,-1,-1,-1,0,0,0.0,1,0.572827,0.557671,0.940444
16,проблем,1,1,1.0,1.0,16.0,0,1,1,-1,1,12,7,-1,-1,-1,3,0,0.0,0,0.510101,0.580226,0.940444
19,частности,1,1,1.0,1.0,19.0,0,4,1,-1,1,15,10,-1,-1,-1,4,0,0.0,1,0.519224,0.432037,0.940444
24,мере,1,1,1.0,2.0,24.0,0,2,1,-1,1,15,12,-1,-1,-1,4,0,0.0,1,0.571194,0.537462,0.940444
25,врач,1,1,1.0,2.0,25.0,1,0,0,-1,1,12,11,-1,-1,-1,0,0,0.0,1,0.586581,0.499800,0.940444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,дереала,2,2,4.0,2.0,239.0,0,1,0,-1,1,21,274,0,0,0,1,0,0.0,1,0.658524,0.468581,0.982723
849,месяц,2,2,5.0,1.0,275.0,0,3,0,-1,1,10,324,-1,-1,-1,5,0,0.0,1,0.607738,0.525600,0.985017
853,манию,2,2,5.0,1.0,279.0,0,3,1,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.529399,0.550035,0.985017
856,мг,2,2,5.0,1.0,282.0,0,0,0,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.553121,0.550035,0.985017


In [1274]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.513224,0.580226,0.940444
217,таблетки,1,1,1.0,21.0,217.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.510001,0.680585,0.940444
295,таблетки,1,1,2.0,0.0,295.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.510001,0.680585,0.987713
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.525554,0.434467,0.987713
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.572827,0.854661,0.987713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15651,вакцина,16,16,16.0,0.0,717.0,0,0,1,-1,1,21,2642,0,0,2,1,2,0.0,1,0.558900,0.426995,0.891408
15692,химиотерапии,16,16,17.0,3.0,758.0,0,1,1,-1,1,21,142,1,0,0,1,2,0.0,1,0.594286,0.569801,0.955551
15751,химиотерапий,16,16,18.0,1.0,817.0,0,1,1,-1,1,21,2609,0,1,1,1,2,0.0,0,0.636680,0.691435,0.943969
15873,вакцина,16,16,19.0,0.0,939.0,0,0,1,-1,1,21,2642,0,0,2,1,2,0.0,1,0.558900,0.426995,0.863237


In [1275]:
# df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])
df.loc[(df['POS'] == 1) & (df['NER'] > 1)]
np.mean(df.loc[(df['POS'] == 1) & (df['NER'] > 1)].CASE)

1.1095238095238096

In [1276]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.620432,0.517389,0.987713
3539,препарат.точнее,3,3,21.0,0.0,1671.0,-1,-1,-1,-1,13,0,691,-1,-1,-1,14,2,3.0,-1,0.634368,0.861262,0.806661
5222,новейший,3,3,38.0,5.0,3354.0,-1,0,0,-1,10,0,1085,-1,0,0,8,2,3350.0,1,0.632377,0.573373,0.982200
6857,тиапридал,5,5,23.0,6.0,659.0,-1,-1,0,-1,0,12,958,-1,-1,-1,0,2,659.0,1,0.678128,0.526667,0.982200
8834,это,7,7,0.0,25.0,295.0,-1,-1,-1,-1,3,12,941,-1,-1,-1,9,3,293.0,-1,0.484618,0.495860,0.984062
8928,хт,7,7,1.0,2.0,389.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.539571,0.493915,0.984062
8987,хт,7,7,1.0,8.0,448.0,-1,-1,-1,-1,-1,21,1651,0,1,2,1,2,0.0,-1,0.539571,0.564514,0.984062
9086,хт,7,7,1.0,13.0,547.0,-1,-1,-1,-1,-1,10,229,-1,-1,-1,5,2,0.0,-1,0.539571,0.575950,0.984062
9497,ксф,7,7,1.0,57.0,958.0,-1,-1,-1,-1,-1,12,1791,-1,-1,-1,0,2,0.0,-1,0.450748,0.659097,0.981763
9935,хт,7,7,7.0,0.0,1396.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.539571,0.493915,0.979373


In [1277]:
med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:2409, :]\
            .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
            loc[:,['TOKEN_VECT', 'CASE', 'HEAD_VECT', 'NER']]


# med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:1409, :]\
#             .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
#             loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]

In [1278]:
med_df

,TOKEN_VECT,CASE,HEAD_VECT,NER
1,0.572827,0,0.557671,0
16,0.510101,1,0.580226,0
19,0.519224,4,0.432037,0
24,0.571194,2,0.537462,0
25,0.586581,0,0.499800,0
...,...,...,...,...
15651,0.558900,0,0.426995,2
15692,0.594286,1,0.569801,2
15751,0.636680,1,0.691435,2
15873,0.558900,0,0.426995,2


In [1279]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)
X, y, X_test, y_test = train.drop(['NER'], axis=1), \
                    train.NER, test.drop(['NER'], axis=1), test.NER

In [1280]:
# pred = knn_clf.predict(X_test)
# pair_eval(pred, y_test)

In [1281]:
X

,TOKEN_VECT,CASE,HEAD_VECT
0,0.562113,0,0.476963
1,0.485996,0,0.594608
2,0.485941,1,0.389474
3,0.572932,0,0.552011
4,0.662116,2,0.580226
...,...,...,...
1828,0.495346,4,0.542085
1829,0.553550,4,0.555371
1830,0.536067,3,0.561613
1831,0.541065,1,0.426673


In [1282]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1828    0
1829    0
1830    0
1831    0
1832    0
Name: NER, Length: 1833, dtype: int64

In [1283]:
len([i for i in y if i == 2])

147

In [1284]:
y_test

0      0
1      0
2      0
3      0
4      2
      ..
781    0
782    0
783    0
784    0
785    0
Name: NER, Length: 786, dtype: int64

In [1285]:
len([i for i in y_test if i == 2])

61

In [1286]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
786  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [1287]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
27  true positive
3  false positive
721  true negative
35  false negative
---------
0.9  presicion
0.43548387096774194  recall
0.5869565217391305  f-score


In [1288]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
32  true positive
15  false positive
709  true negative
30  false negative
---------
0.6808510638297872  presicion
0.5161290322580645  recall
0.5871559633027522  f-score
